In [129]:
from draco.learn import linear
from draco.learn import data_util
from draco.run import run
from draco.spec import Task, Query
import numpy as np
import pandas as pd
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split

%matplotlib inline
%config InlineBackend.figure_format = 'retina'

In [130]:
test_size = 0.3

In [131]:
data, _ = data_util.load_data(test_size=test_size)
data = data[data.source == 'younghoon']

raw_data = data_util.load_neg_pos_specs()

In [132]:
X = data.positive - data.negative
y = np.ones(len(X))

In [133]:
clf = linear.train_model(X, test_size=test_size)
pred = clf.predict(X)

Train score:  0.9273049645390071
Dev score:  0.9259259259259259


In [134]:
weights = clf.coef_[0]
names = X.columns

In [176]:
const_dict = dict(zip(map(lambda x: f'{x}_weight', names), map(int, weights*1000)))
weight_dict = dict(zip(names, weights))

In [177]:
only_correct = X[pred == 1]

len(only_correct), len(data)

(748, 807)

In [180]:
files = ['define.lp', 'test.lp', 'features.lp', 'count.lp', 'output.lp']

def aggregate(violations):
    ret = 0.0
    for n, v in violations.items():
        ret += weight_dict[n] * v
    return ret

for i in range(len(only_correct))[:10]:
    entry = only_correct.iloc[i]
    name = entry.name
    pair = raw_data[name]

    task_n = Task(pair.data, Query.from_vegalite(pair.negative))
    task_p = Task(pair.data, Query.from_vegalite(pair.positive))
    
    print(clf.predict([entry]))
    # print(clf.coef_.dot(np.array([entry]).T))

    print(name)
    
#     cn = run(task_n, constants=const_dict).cost
#     cp = run(task_p, constants=const_dict).cost
#     print(cn, cp, cn > cp)
    
    cn = aggregate(run(task_n, files=files, silence_warnings=True).violations)
    cp = aggregate(run(task_p, files=files, silence_warnings=True).violations)
    
    print(cn, cp, cn > cp)
    
    print

[1.]
younghoon-664
-0.0014407701675249879 0.6769377813650127 False
[1.]
younghoon-955
-0.472443278575694 0.571969391032046 False
[1.]
younghoon-1016
-0.472443278575694 0.2398446423912579 False
[1.]
younghoon-843
-0.472443278575694 0.2398446423912579 False
[1.]
younghoon-327
0.4470663108482781 0.31645835234652875 True
[1.]
younghoon-1043
-0.07262639522718228 1.0823050305300144 False
[1.]
younghoon-991
-0.3468400781199943 1.0823050305300144 False
[1.]
younghoon-689
0.34209792051531135 0.21148996201356202 True
[1.]
younghoon-1080
-0.664739200634048 0.08914720336851262 False
[1.]
younghoon-835
0.48896408671702435 0.8900091084716604 False
